In [2]:
raw_fname = "IoU_FTSmartAudit_datasets.json"

In [3]:
dataset = [
    {"dataset_name":"IoU_FTSmartAudit_datasets", 
     "vulnerability": "Integer overflow/underflow"}
    ]

end_sample_num = 10

In [33]:
def read_json_files(fewshot_dir, loc_dir):
    all_data = []
    for filename in os.listdir(fewshot_dir):
        if filename.endswith(".json"):
            file_path = os.path.join(fewshot_dir, filename)
            with open(file_path, 'r') as f:
                data = json.load(f)
            with open(os.path.join(loc_dir, filename), 'r') as f:
                fewshot_helper = json.load(f)
                # print("FEWSHOT HELPER")
                # print(fewshot_helper)
            data.append({"helper": fewshot_helper})
            # print("FS DATA")
            # print(data)
            all_data.append(data)
    return all_data
def get_numbered_contract(contract):
    lines = contract.split("\n")
    numbered_lines = [f"{i+1}: {line}" for i, line in enumerate(lines)]
    return "\n".join(numbered_lines)

In [34]:
from pydantic import BaseModel, Field
from typing import List

class SingleVulnerability(BaseModel):
    vulnerableLines: str
    vulnerableCode: List[str]
    vulnerabilityReason: str=Field(description='Why the code snippet is vulnerable') 
    potentialSecurityRisk: str=Field(description='The potential security risk the vulnerability may cause.') 
    fixedCode: str=Field(description='The healthy code snippet, not the way to fix ') 

class FullVulnerability(BaseModel):
    vulnerabilities: List[SingleVulnerability]



dataset_name = dataset[0]["dataset_name"]
vulnerability =dataset[0]["vulnerability"]
raw_fname = os.path.join("..", "..", "..","data", "dataset", "raw", dataset_name+".json")
fewshot_dir = os.path.join("..", "..", "..","data", "dataset", "few_shots", dataset_name)
processed_dir = os.path.join("..", "..", "..","data", "dataset", "processed_data", dataset_name)
loc_dir = os.path.join("..", "..", "..","data", "processed_data", dataset_name, "LOCs")
os.makedirs(loc_dir, exist_ok=True)
loc_helper_dir = os.path.join("..", "..", "..","data", "processed_data", dataset_name, "LOCs_old")

fewshot_data = read_json_files(fewshot_dir, loc_helper_dir)

In [35]:
with open(raw_fname, 'r') as f:
    raw_data = json.load(f)

In [38]:
print(get_numbered_contract(raw_data[0]['Source Code']))

1: /*
2:  * @source: https://github.com/sigp/solidity-security-blog
3:  * @author: Steve Marx
4:  * @vulnerable_at_lines: 20,22
5:  */
6: 
7:  pragma solidity ^0.4.18;
8: 
9:  contract Token {
10: 
11:    mapping(address => uint) balances;
12:    uint public totalSupply;
13: 
14:    function Token(uint _initialSupply) {
15:      balances[msg.sender] = totalSupply = _initialSupply;
16:    }
17: 
18:    function transfer(address _to, uint _value) public returns (bool) {
19:      // <yes> <report> ARITHMETIC
20:      require(balances[msg.sender] - _value >= 0);
21:      // <yes> <report> ARITHMETIC
22:      balances[msg.sender] -= _value;
23:      balances[_to] += _value;
24:      return true;
25:    }
26: 
27:    function balanceOf(address _owner) public constant returns (uint balance) {
28:      return balances[_owner];
29:    }
30:  }


In [39]:
print(get_numbered_contract(raw_data[0]['input']))

1: pragma solidity ^0.4.18;
2: 
3:  contract Token {
4: 
5:    mapping(address => uint) balances;
6:    uint public totalSupply;
7: 
8:    function Token(uint _initialSupply) {
9:      balances[msg.sender] = totalSupply = _initialSupply;
10:    }
11: 
12:    function transfer(address _to, uint _value) public returns (bool) {
13:      
14:      require(balances[msg.sender] - _value >= 0);
15:      
16:      balances[msg.sender] -= _value;
17:      balances[_to] += _value;
18:      return true;
19:    }
20: 
21:    function balanceOf(address _owner) public constant returns (uint balance) {
22:      return balances[_owner];
23:    }
24:  }
